In [2]:
import pandas as pd
import numpy as np
import sys
import time
import polars as pl
import requests
import os

pd.set_option("display.max_column",None)
print(os.getcwd())

/Volumes/Backup/OverperformxG/OverperformXG


In [42]:
os.getcwd()

'/Volumes/Backup/OverperformxG/OverperformXG'

# API Football:
https://www.api-football.com

In [37]:
# API Key and Configs

headers_rapid_api = {
	"X-RapidAPI-Key": "64f82034damsh6dd2baeda9216d1p173668jsnddc6687707f5",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}
headers_api_sport = {
	"x-apisports-key": "0a078f979ea1996d42df390199afe0cf",
	"X-apisports-Host": "https://v3.football.api-sports.io/"
}


In [46]:
leagues_dat = pd.read_parquet("/Volumes/Backup/OverperformxG/OverperformXG/data/Leagues/leagues.parquet")
leagues_dat[['league.id','league.name','country.name']]

,league.id,league.name,country.name
0,4,Euro Championship,World
1,21,Confederations Cup,World
2,61,Ligue 1,France
3,144,Jupiler Pro League,Belgium
4,71,Serie A,Brazil
...,...,...,...
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World
1130,1133,Goiano - 3,Brazil
1131,1001,CONCACAF Women U20,World


In [57]:
# Configs
teams = ["Liverpool","Wolves"] # teams to pull players data of
seasons = ["2022","2021","2023"] # seasons to pull players and teams stats of
league_dict_subset = league_dict[league_dict['league.name'].isin(["Premier League","La Liga","Serie A"]) & league_dict['country.name'].isin(['England',"Spain","Italy"])] # league ID to pull from, current values: {39:premier league}, Add to dictionary as needed

In [58]:
league_dict_subset

,league.id,league.name,country.name
5,39,Premier League,England
7,135,Serie A,Italy
10,140,La Liga,Spain


In [63]:
# Fixtures for Events Data
## Main Entry point to get match summary and drill down with fixture ids
fixtures_dat = dict()
for league_name,league in league_dict.items():
    for season in seasons:
        print("for league and season",league,"_",season)
        url = "https://v3.football.api-sports.io/fixtures?league={}&season={}".format(league,season)
        fixtures_response = requests.get(url,headers=headers_api_sport)
        fixtures_dat[league_name + "_" + season ] = pd.json_normalize(fixtures_response.json()['response'])
        fixtures_dat[league_name + "_" + season ].to_parquet("./data/" + league_name + "_" + season + ".parquet")
        

for league and season 39 _ 2022
for league and season 39 _ 2021
for league and season 39 _ 2023


In [30]:
# All teams
team_id_map = fixtures_dat['Premier_League_2022'][['teams.home.name','teams.home.id']]
team_id_map

,teams.home.name,teams.home.id
0,Crystal Palace,52
1,Fulham,36
2,Bournemouth,35
3,Leeds,63
4,Leicester,46
...,...,...
375,Everton,45
376,Leeds,63
377,Leicester,46
378,Manchester United,33


In [31]:
# Polars tests
fixtures_pl = pl.DataFrame(fixtures_dat['Premier_League_2022'])

In [60]:
# Save fixtures data to disk

fixtures_pl.write_parquet("./data/"+league+".parquet")

In [32]:
fixtures_pl

fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
i64,str,str,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,str,str,str,i64,str,i64,str,str,bool,i64,str,str,bool,i64,i64,i64,i64,i64,i64,str,str,str,str
867946,"""A. Taylor""","""UTC""","""2022-08-05T19:00:00+00:00""",1659726000,1659726000,1659729600,525,"""Selhurst Park""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",52,"""Crystal Palace""","""https://media.api-sports.io/fo…",false,42,"""Arsenal""","""https://media.api-sports.io/fo…",true,0,2,0,1,0,2,null,null,null,null
867947,"""A. Madley""","""UTC""","""2022-08-06T11:30:00+00:00""",1659785400,1659785400,1659789000,535,"""Craven Cottage""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",36,"""Fulham""","""https://media.api-sports.io/fo…",null,40,"""Liverpool""","""https://media.api-sports.io/fo…",null,2,2,1,0,2,2,null,null,null,null
867948,"""P. Bankes""","""UTC""","""2022-08-06T14:00:00+00:00""",1659794400,1659794400,1659798000,504,"""Vitality Stadium""","""Bournemouth, Dorset""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",35,"""Bournemouth""","""https://media.api-sports.io/fo…",true,66,"""Aston Villa""","""https://media.api-sports.io/fo…",false,2,0,1,0,2,0,null,null,null,null
867949,"""R. Jones""","""UTC""","""2022-08-06T14:00:00+00:00""",1659794400,1659794400,1659798000,546,"""Elland Road""","""Leeds, West Yorkshire""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",63,"""Leeds""","""https://media.api-sports.io/fo…",true,39,"""Wolves""","""https://media.api-sports.io/fo…",false,2,1,1,1,2,1,null,null,null,null
867950,"""J. Gillett""","""UTC""","""2022-08-07T13:00:00+00:00""",1659877200,1659877200,1659880800,547,"""King Power Stadium""","""Leicester, Leicestershire""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",46,"""Leicester""","""https://media.api-sports.io/fo…",null,55,"""Brentford""","""https://media.api-sports.io/fo…",null,2,2,1,0,2,2,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
868321,"""S. Attwell""","""UTC""","""2023-05-28T15:30:00+00:00""",1685287800,1685287800,1685291400,8560,"""Goodison Park""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 38""",45,"""Everton""","""https://media.api-sports.io/fo…",true,35,"""Bournemouth""","""https://media.api-sports.io/fo…",false,1,0,0,0,1,0,null,null,null,null
868322,"""A. Taylor""","""UTC""","""2023-05-28T15:30:00+00:00""",1685287800,1685287800,1685291400,546,"""Elland Road""","""Leeds, West Yorkshire""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Seaso

In [51]:
# Select columns
fixtures_pl.select(pl.col(["fixture.referee","fixture.date"]))

# chain function
fixtures_pl.filter(pl.col("teams.home.name") == "Liverpool").group_by(pl.col("teams.home.winner")).agg([pl.col("teams.home.name").count().alias("sum_games"),pl.col(["goals.home","goals.away"]).mean()])

teams.home.winner,sum_games,goals.home,goals.away
bool,u32,f64,f64
true,13,2.923077,0.615385
false,1,1.0,2.0
null,5,1.4,1.4


In [50]:
fixtures_pl.filter(pl.col("teams.home.name") == "Liverpool").shape

(19, 38)

In [56]:
fixtures_pl.filter((pl.col("teams.home.name") == "Liverpool") | (pl.col("teams.away.name") == "Liverpool"))

fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
i64,str,str,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,str,str,str,i64,str,i64,str,str,bool,i64,str,str,bool,i64,i64,i64,i64,i64,i64,str,str,str,str
867947,"""A. Madley""","""UTC""","""2022-08-06T11:30:00+00:00""",1659785400,1659785400,1659789000,535,"""Craven Cottage""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",36,"""Fulham""","""https://media.api-sports.io/fo…",null,40,"""Liverpool""","""https://media.api-sports.io/fo…",null,2,2,1,0,2,2,null,null,null,null
867961,"""P. Tierney""","""UTC""","""2022-08-15T19:00:00+00:00""",1660590000,1660590000,1660593600,550,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 2""",40,"""Liverpool""","""https://media.api-sports.io/fo…",null,52,"""Crystal Palace""","""https://media.api-sports.io/fo…",null,1,1,0,1,1,1,null,null,null,null
867972,"""M. Oliver""","""UTC""","""2022-08-22T19:00:00+00:00""",1661194800,1661194800,1661198400,556,"""Old Trafford""","""Manchester""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 3""",33,"""Manchester United""","""https://media.api-sports.io/fo…",true,40,"""Liverpool""","""https://media.api-sports.io/fo…",false,2,1,1,0,2,1,null,null,null,null
867981,"""S. Attwell""","""UTC""","""2022-08-27T14:00:00+00:00""",1661608800,1661608800,1661612400,550,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 4""",40,"""Liverpool""","""https://media.api-sports.io/fo…",true,35,"""Bournemouth""","""https://media.api-sports.io/fo…",false,9,0,5,0,9,0,null,null,null,null
867994,"""A. Marriner""","""UTC""","""2022-08-31T19:00:00+00:00""",1661972400,1661972400,1661976000,550,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 5""",40,"""Liverpool""","""https://media.api-sports.io/fo…",true,34,"""Newcastle""","""https://media.api-sports.io/fo…",false,2,1,0,1,2,1,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
868283,"""P. Tierney""","""UTC""","""2023-04-30T15:30:00+00:00""",1682868600,1682868600,1682872200,550,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 34""",40,"""Liverpool""","""https://media.api-sports.io/fo…",true,47,"""Tottenham""","""https://media.api-sports.io/fo…",false,4,3,3,1,4,3,null,null,null,null
868289,"""A. Taylor""","""UTC""","""2023-05-06T16:30:00+00:00""",1683390600,1683390600,1683394200,550,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 35""",40,"""Liverpool""","""https://media.api-sports.io/fo…

In [31]:
all_fixtures = pd.DataFrame()
for fixture in os.listdir(os.getcwd() + "/data"):
    data = pd.read_parquet(os.getcwd() + "/data/" + fixture)
    print(data.head())
    all_fixtures = pd.concat([all_fixtures,data])
    

   fixture.id fixture.referee fixture.timezone               fixture.date  \
0      867946       A. Taylor              UTC  2022-08-05T19:00:00+00:00   
1      867947       A. Madley              UTC  2022-08-06T11:30:00+00:00   
2      867948       P. Bankes              UTC  2022-08-06T14:00:00+00:00   
3      867949        R. Jones              UTC  2022-08-06T14:00:00+00:00   
4      867950      J. Gillett              UTC  2022-08-07T13:00:00+00:00   

   fixture.timestamp  fixture.periods.first  fixture.periods.second  \
0         1659726000             1659726000              1659729600   
1         1659785400             1659785400              1659789000   
2         1659794400             1659794400              1659798000   
3         1659794400             1659794400              1659798000   
4         1659877200             1659877200              1659880800   

   fixture.venue.id  fixture.venue.name         fixture.venue.city  \
0               525       Selhurst Park 

In [34]:
all_fixtures = pl.DataFrame(all_fixtures)

In [35]:
# Once All fixtures are loaded,
# Get team data


fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
i64,str,str,str,i64,i64,i64,f64,str,str,str,str,i64,i64,str,str,str,str,i64,str,i64,str,str,bool,i64,str,str,bool,i64,i64,i64,i64,i64,i64,str,str,str,str
867946,"""A. Taylor""","""UTC""","""2022-08-05T19:00:00+00:00""",1659726000,1659726000,1659729600,525.0,"""Selhurst Park""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",52,"""Crystal Palace""","""https://media.api-sports.io/fo…",false,42,"""Arsenal""","""https://media.api-sports.io/fo…",true,0,2,0,1,0,2,null,null,null,null
867947,"""A. Madley""","""UTC""","""2022-08-06T11:30:00+00:00""",1659785400,1659785400,1659789000,535.0,"""Craven Cottage""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",36,"""Fulham""","""https://media.api-sports.io/fo…",null,40,"""Liverpool""","""https://media.api-sports.io/fo…",null,2,2,1,0,2,2,null,null,null,null
867948,"""P. Bankes""","""UTC""","""2022-08-06T14:00:00+00:00""",1659794400,1659794400,1659798000,504.0,"""Vitality Stadium""","""Bournemouth, Dorset""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",35,"""Bournemouth""","""https://media.api-sports.io/fo…",true,66,"""Aston Villa""","""https://media.api-sports.io/fo…",false,2,0,1,0,2,0,null,null,null,null
867949,"""R. Jones""","""UTC""","""2022-08-06T14:00:00+00:00""",1659794400,1659794400,1659798000,546.0,"""Elland Road""","""Leeds, West Yorkshire""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",63,"""Leeds""","""https://media.api-sports.io/fo…",true,39,"""Wolves""","""https://media.api-sports.io/fo…",false,2,1,1,1,2,1,null,null,null,null
867950,"""J. Gillett""","""UTC""","""2022-08-07T13:00:00+00:00""",1659877200,1659877200,1659880800,547.0,"""King Power Stadium""","""Leicester, Leicestershire""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2022,"""Regular Season - 1""",46,"""Leicester""","""https://media.api-sports.io/fo…",null,55,"""Brentford""","""https://media.api-sports.io/fo…",null,2,2,1,0,2,2,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1035549,"""D. Bond""","""UTC""","""2024-05-19T15:00:00+00:00""",1716130800,1716130800,1716134400,525.0,"""Selhurst Park""","""London""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2023,"""Regular Season - 38""",52,"""Crystal Palace""","""https://media.api-sports.io/fo…",true,66,"""Aston Villa""","""https://media.api-sports.io/fo…",false,5,0,2,0,5,0,null,null,null,null
1035550,"""C. Kavanagh""","""UTC""","""2024-05-19T15:00:00+00:00""",1716130800,1716130800,1716134400,550.0,"""Anfield""","""Liverpool""","""Match Finished""","""FT""",90,39,"""Premier League""","""England""","""https://media.api-sports.io/fo…","""https://media.api-sports.io/fl…",2023,"""Regular Sea